<a href="https://colab.research.google.com/github/Evan-eng/anthropic_superposition_experiment_recreation/blob/main/demonstrating_superposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt


In [ ]:
features = 100 # n
hidden = 20 # m

class linear_model(nn.Module):
  # Create model here
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(n, m, bias=False)
    self.layer2 = nn.Linear(m, n)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    return x

# h = Wx
# (m x 1) = (m x n) (n x 1)

# x' = W^T h + b
# (n x 1) = (n x m) (m x 1) + (n x 1)

class relu_model(nn.Module):
  # Create model here
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(n, m, bias=False)
    self.layer2 = nn.Linear(m, n)

    self.relu = nn.ReLu()

  def forward(self, x):
    x = self.relu(self.layer1(x))
    x = self.relu(self.layer2(x))
    return x
